# Transfer learning with TensorFlow part 1 : Feature extraction
There are two main benefits to using transfer learning:

Can leverage an existing neural network architecture proven to work on problems similar to our own.
Can leverage a working neural network architecture which has already learned patterns on similar data to our own. This often results in achieving great results with less custom data.

In [1]:
# Are we using a GPU
!nvidia-smi

Sun Dec 25 08:05:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download and become one with the data

In [3]:
# Get 10 percent of ten food classes from Food101
import zipfile

# Download the data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close

--2022-12-25 08:10:42--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.128, 142.251.167.128, 142.250.188.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   284MB/s    in 0.6s    

2022-12-25 08:10:43 (284 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



<bound method ZipFile.close of <zipfile.ZipFile filename='10_food_classes_10_percent.zip' mode='r'>>

In [4]:
# How many images in each folder
import os

# Walk through 10 percent data directory and list number of file
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in ' {dirpath} '.")

There are 2 directories and 0 images in ' 10_food_classes_10_percent '.
There are 10 directories and 0 images in ' 10_food_classes_10_percent/test '.
There are 0 directories and 250 images in ' 10_food_classes_10_percent/test/steak '.
There are 0 directories and 250 images in ' 10_food_classes_10_percent/test/ice_cream '.
There are 0 directories and 250 images in ' 10_food_classes_10_percent/test/hamburger '.
There are 0 directories and 250 images in ' 10_food_classes_10_percent/test/chicken_wings '.
There are 0 directories and 250 images in ' 10_food_classes_10_percent/test/chicken_curry '.
There are 0 directories and 250 images in ' 10_food_classes_10_percent/test/pizza '.
There are 0 directories and 250 images in ' 10_food_classes_10_percent/test/grilled_salmon '.
There are 0 directories and 250 images in ' 10_food_classes_10_percent/test/ramen '.
There are 0 directories and 250 images in ' 10_food_classes_10_percent/test/sushi '.
There are 0 directories and 250 images in ' 10_food_

## Creating data loaders (preparing the data)
we will use the ImageDataGenerator Class to load in our data

In [7]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMAGE_SHAPE =(224,224)
BATCH_SIZE = 32
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"
train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)
test_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                         target_size = IMAGE_SHAPE,
                                                         batch_size = BATCH_SIZE,
                                                         class_mode = "categorical")
print("Testing images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size = IMAGE_SHAPE,
                                             batch_size = BATCH_SIZE,
                                             class_mode = "categorical")


Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


## Setting up call backs (things to run whilst our model trains)
callbacks are extra functionality you can add to your mdoels
to be performed during or after training Some of the most popular callabacks are:
* Tracking experiments with the TensorBoard callback
* Model checkpoint with the ModelCheckpoint callback
* Stopping a model from training (before it trains too long and overfits) with the EarlyStopping callbacks 

In [8]:
#  Create tensorboard call back and functionize it
import datetime

def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
  tensorboard_callback = tf.keras.callabacks.TensorBoard(log_dir = logd_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return(tensorboard_callback)

## Creating model using tensorflow hub
In the past we have used tensor flow to write our models line by line.
Now we are going to do a similar process except the majority of our model layers are going to come from tensorflow hub.

We can access  pretrained models on https://tfhub.dev/